In [1]:
import tqdm
import numpy
import pandas
import scipy.spatial

In [2]:
numpy.random.seed(42)

In [3]:
CROSSOVER_PROBABILITY = 0.9
MUTATION_PROBABILITY = 0.5
POPULATION_SIZE = 10
GENERATIONS = 10000000
GENES = 100

In [4]:
def evaluate_chromosome(chromosome, optim):
    # number of common elements between chromosomes
    return len(numpy.where(chromosome == optim)[0])

In [5]:
# returns index of elite chromosome
def get_elite_chromosome(population, optim):
    idx, max_val = 0, 0
    for i in range(population.shape[0]):
        val = evaluate_chromosome(population[i], optim)
        if max_val < val:
            idx = i
            max_val = val
    return idx

In [6]:
def gauss_replacement(chromosome):
    # define number of genes
    idx_interval = chromosome.shape[0]
    # generate random gaussian distribution
    GAUSSIAN = numpy.random.normal(loc=3, scale=2.0, size=chromosome.shape[0]).astype(numpy.int64)
    # fix lower numbers
    GAUSSIAN[GAUSSIAN < 1] = 1
    # fix higher numbers
    GAUSSIAN[GAUSSIAN > 5] = 5
    # define mutated array
    mutated_chromosome = numpy.zeros(idx_interval, dtype=numpy.int64)
    # define random generator for gene mutation decision
    SEQUENCE = numpy.random.uniform(0, 1, idx_interval)
    for i in range(idx_interval):
        # mutate gene with respect to current SEQUENCE probability
        if SEQUENCE[i] > 0.5:
            # probability valid, MUTATE
            mutated_chromosome[i] = GAUSSIAN[i]
        else:
            # probability inadequate, NO MUTATION
            mutated_chromosome[i] = chromosome[i]
    return mutated_chromosome

In [7]:
def elitism(population, m_probability, optim):
    SEQUENCE = numpy.random.uniform(0, 1, population.shape[0])
    mutated_population, mutated_chromosome = None, None
    elite_chromosome_idx = get_elite_chromosome(population, optim)
    for i in range(population.shape[0]):
        chromosome = population[i]
        if SEQUENCE[i] < m_probability and i != elite_chromosome_idx:
            # mutate chromosome
            mutated_chromosome = gauss_replacement(chromosome)
            # append chromosomes to the mutated population
            if i == 0:
                # if loop run for first time, then initialize the generation population
                mutated_population = mutated_chromosome
            else:
                # after first time, stack chromosomes to the generation population
                mutated_population = numpy.vstack((mutated_population, mutated_chromosome))
        else:
            # NO mutation
            # append chromosomes to the mutated population
            if i == 0:
                # if loop run for first time, then initialize the generation population
                mutated_population = chromosome
            else:
                # after first time, stack chromosomes to the generation population
                mutated_population = numpy.vstack((mutated_population, chromosome))
    return mutated_population

In [8]:
# function that converts parent indeces to the actual population: returns parent pairs [for 10 neighbors: outputs matrix(2x5)]
def index_to_chromosome_decode(idx, population):
    # reshape indeces to handle them easier [with population size = 10, idx is reshaped to: matrix(5,2)]
    idx = idx.reshape(int(population.shape[0]/2), 2)
    # define the generation population
    generation = None
    # convert given indeces to chromosomes 
    for i in range(int(population.shape[0]/2)):
        # extract 2 indeces
        X, Y = population[idx[i]]
        # stack the chromosomes
        if i == 0:
            # if loop run for first time, then initialize the generation population
            generation = numpy.stack((X, Y))
        else:
            # after first time, stack chromosomes to the generation population
            generation = numpy.vstack((generation, numpy.stack((X, Y))))
    # reshape matrix
    population = numpy.zeros((int(generation.shape[0]/2), 2, generation[0].shape[0]), dtype=numpy.int64)
    for i in range(population.shape[0]):
        for j in range(population.shape[1]):
            population[i][j] = generation[i * 2 + j]
    return population

In [9]:
def tournament_selection(population, optim):
    # initialize random sequence
    SEQUENCE = numpy.random.uniform(0, 1, population.shape[0])
    # get pearsons
    pearsons = numpy.empty((population.shape[0]), dtype=numpy.int64)
    pearsons = numpy.fromiter((evaluate_chromosome(population[i], optim) for i in range(population.shape[0])), pearsons.dtype)
    # get parents
    parents = numpy.empty(population.shape[0], dtype=numpy.int64)
    for i in range(population.shape[0]):
        k = numpy.ceil(SEQUENCE[i] * 10).astype(numpy.int64)
        chromosome_pointers = numpy.random.choice(numpy.arange(population.shape[0]), k)
        evaluation = pearsons[chromosome_pointers].max()
        if len(numpy.where(pearsons == evaluation)[0]) > 1:
            parents[i] = numpy.where(pearsons == evaluation)[0][0]
        else:
            parents[i] = numpy.where(pearsons == evaluation)[0]
    return index_to_chromosome_decode(parents, population)

In [10]:
def multiple_point_crossover(parent_pairs, x_probability):
    # define random probability sequence
    SEQUENCE = numpy.random.uniform(0, 1, parent_pairs.shape[0])
    # define new generation population variable
    population_hat = None
    # perform single point crossover 
    for i in range(parent_pairs.shape[0]):
        X, Y = parent_pairs[i]
        # define max index boundary
        chromosome_shape = X.shape[0]
        # initialize new chromosome
        a, b = numpy.zeros((2, chromosome_shape), dtype=numpy.int64)
        # crossover random point
        x_idx, y_idx = numpy.sort(numpy.random.randint(0, chromosome_shape, 2))
        # first child chromosome
        a = numpy.concatenate((X[ :x_idx], Y[x_idx:y_idx], X[y_idx: ]))
        # second child chromosome
        b = numpy.concatenate((Y[ :x_idx], X[x_idx:y_idx], Y[y_idx: ]))
        # crossover with respect to the crosover probability
        if SEQUENCE[i] < x_probability:
            # append children to form the new population
            if i == 0:
                # if loop run for first time, then initialize the generation population
                population_hat = numpy.stack((a, b))
            else:
                # after first time, stack chromosomes to the generation population
                population_hat = numpy.vstack((population_hat, numpy.stack((a, b))))
        else:
            # append parents to the new population
            if i == 0:
                # if loop run for first time, then initialize the generation population
                population_hat = numpy.stack((X, Y))
            else:
                # after first time, stack chromosomes to the generation population
                population_hat = numpy.vstack((population_hat, numpy.stack((X, Y))))
    return population_hat

In [11]:
# define optim
optim = numpy.random.choice((numpy.arange(5)+1), GENES)

In [12]:
# define population (size 20x20)
population = numpy.random.choice(numpy.unique(optim), (POPULATION_SIZE, optim.shape[0]))

In [13]:
population_hat = population

In [14]:
# create GA fit loop
for gen in tqdm.tqdm_notebook(range(GENERATIONS)):
    parent_pairs = tournament_selection(population_hat, optim)
    population_hat = multiple_point_crossover(parent_pairs, CROSSOVER_PROBABILITY)
    population_hat = elitism(population_hat, MUTATION_PROBABILITY, optim)
    universe_evaluation = numpy.fromiter((evaluate_chromosome(population_hat[i], optim) for i in range(population_hat.shape[0])), numpy.int64)
    if universe_evaluation.max() > 95:
        break;

/home/andreas/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


In [15]:
similarity = numpy.zeros(population_hat.shape[0], dtype=numpy.int64)
evaluations = numpy.zeros(population_hat.shape[0], dtype=numpy.int64)
mse = numpy.zeros(population_hat.shape[0], dtype=numpy.int64)

In [16]:
for i in range(similarity.shape[0]):
    similarity[i] = 1 - scipy.spatial.distance.cosine(optim, population_hat[i])
    evaluations[i] = evaluate_chromosome(population_hat[i], optim)
    mse[i] = (numpy.square(optim - population_hat[i])).mean(axis=None)

In [17]:
similarity.max()

0

In [18]:
mse.min()

2

In [19]:
evaluations.max()

67

In [20]:
optim

array([4, 5, 3, 5, 5, 2, 3, 3, 3, 5, 4, 3, 5, 2, 4, 2, 4, 5, 1, 4, 2, 5,
       4, 1, 1, 3, 3, 2, 4, 4, 3, 4, 4, 1, 3, 5, 3, 5, 1, 2, 4, 1, 4, 2,
       2, 1, 2, 5, 2, 4, 4, 4, 4, 5, 3, 1, 4, 2, 4, 2, 2, 4, 5, 2, 2, 4,
       2, 2, 4, 4, 1, 5, 5, 2, 5, 2, 1, 4, 4, 4, 5, 1, 5, 5, 1, 1, 1, 1,
       4, 3, 3, 1, 3, 3, 1, 3, 5, 2, 2, 1])

In [21]:
population_hat[0]

array([4, 2, 3, 5, 5, 1, 3, 3, 3, 1, 4, 3, 5, 2, 4, 2, 1, 2, 1, 4, 2, 4,
       4, 1, 1, 3, 3, 2, 1, 3, 3, 4, 5, 3, 3, 5, 3, 2, 1, 2, 5, 1, 4, 4,
       2, 1, 2, 1, 4, 1, 4, 4, 4, 1, 3, 1, 4, 4, 4, 2, 5, 4, 1, 1, 2, 2,
       3, 2, 1, 4, 1, 5, 4, 2, 5, 2, 1, 3, 1, 4, 5, 5, 5, 5, 3, 1, 1, 1,
       4, 3, 3, 1, 3, 3, 1, 4, 2, 2, 3, 3])

In [22]:
universe_evaluation

array([67, 40, 59, 59, 67, 42, 36, 67, 67, 38])